# Performance Evaluation: Sieve of Eratosthenes (MPI)

This notebook automates the performance evaluation described in Chapter 4 (visit /doc/documentation.pdf). It performs the following steps:

1.  **Environment Check**: Verifies the system hardware and MPI version.
2.  **Run Experiments**: Executes the MPI program (`bin/program`) for different problem sizes (`N`) and process counts (`p`).
3.  **Collect Data**: Runs each experiment multiple times (`REPETITIONS`) and calculates the average execution time, parsing the output from the C program.
4.  **Process Data**: Calculates Speedup ($S = T_s / T_p$) and Efficiency ($E = S / p$).
5.  **Generate Outputs**: Displays the final data as a Markdown table (matching Table 4.2) and generates the three required plots for the graphical analysis.

## 1. Imports and Configuration

Here we'll import the necessary libraries and define the experimental parameters.

In [ ]:
import subprocess
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Configuration ---

# Path to your MPI executable
# (Assumes 'sieve' is the MPI version from your Makefile)
PROGRAM_PATH = "./bin/sieve"

# Problem sizes (N)
N_VALUES = [10**6, 10**7, 10**8]

# Process counts (p)
P_VALUES = [1, 2, 4, 6, 8, 10]

# Number of times to repeat each experiment for a stable average
REPETITIONS = 5

# Set plot style
sns.set_theme(style="whitegrid")

## 3. Experiment Execution

First, we define a helper function to run the C program and parse its output. We'll use a regular expression to find the execution time, as described in your `sieve_sequential.c` implementation.

**Note**: This cell will take a long time to run, especially for $N = 10^8$.

In [ ]:
def run_experiment(p, n):
    """Runs the MPI program and parses its output for execution time."""
    command = ["mpirun", "-np", str(p), PROGRAM_PATH, str(n)]
    
    try:
        # Execute the command
        result = subprocess.run(
            command, 
            capture_output=True, 
            text=True, 
            timeout=600  # 10-minute timeout for large runs
        )
        
        if result.returncode != 0:
            print(f"Error running {command}: {result.stderr}")
            return None

        # Parse the output
        # This regex looks for the line from your C code's output
        match = re.search(r"Execution time \(computation only\): (\d+\.\d+) seconds", result.stdout)
        
        if match:
            return float(match.group(1))
        else:
            print(f"Could not parse time from output: {result.stdout}")
            return None
            
    except Exception as e:
        print(f"An error occurred with {command}: {e}")
        return None

# --- Main Execution Loop ---

results = []
print(f"Starting experiments... Program: {PROGRAM_PATH}")
print(f"N values: {N_VALUES}")
print(f"p values: {P_VALUES}")
print(f"Repetitions: {REPETITIONS}\n")

if not os.path.exists(PROGRAM_PATH):
    print(f"ERROR: Program not found at {PROGRAM_PATH}")
    print("Please compile the project first (e.g., 'make all')")
else:
    for n in N_VALUES:
        for p in P_VALUES:
            times = []
            print(f"Running: N={n:<9} p={p:<2}... ", end="")
            
            for i in range(REPETITIONS):
                time = run_experiment(p, n)
                if time is not None:
                    times.append(time)
            
            if times:
                avg_time = np.mean(times)
                print(f"Avg: {avg_time:.4f}s")
                results.append({
                    'N': n,
                    'Processes (p)': p,
                    'Parallel Time (Tp) [s]': avg_time
                })
            else:
                print("Failed.")

print("\nAll experiments complete.")

## 4. Data Processing and Table Generation

Now, we'll process the raw timing data into a `pandas` DataFrame to calculate Speedup and Efficiency. The sequential time ($T_s$) is the time measured for $p=1$.

In [ ]:
# Convert results list to a DataFrame
df = pd.DataFrame(results)

# Extract Sequential Time (Ts) for each N (which is the Tp where p=1)
df_ts = df[df['Processes (p)'] == 1][['N', 'Parallel Time (Tp) [s]']]
df_ts = df_ts.rename(columns={'Parallel Time (Tp) [s]': 'Sequential Time (Ts) [s]'})

# Merge Ts back into the main DataFrame
df = pd.merge(df, df_ts, on='N')

# Calculate Speedup and Efficiency
df['Speedup (S)'] = df['Sequential Time (Ts) [s]'] / df['Parallel Time (Tp) [s]']
df['Efficiency (E)'] = df['Speedup (S)'] / df['Processes (p)']

# Reorder columns to match your LaTeX table
df = df[[
    'N', 
    'Processes (p)', 
    'Sequential Time (Ts) [s]', 
    'Parallel Time (Tp) [s]', 
    'Speedup (S)', 
    'Efficiency (E)'
]]

print("--- Execution Results Table (for LaTeX) ---")
# Print as a Markdown table for easy copy-pasting
print(df.to_markdown(index=False, floatfmt=".4f"))

## 5. Graphical Analysis

Here we generate the plots for your report.

### Figure 1: Comparison of Execution Times

*(Corresponds to your `fig:time_comparison`)*

In [ ]:
plt.figure(figsize=(10, 6))
g = sns.lineplot(
    data=df, 
    x='Processes (p)', 
    y='Parallel Time (Tp) [s]', 
    hue='N', 
    style='N', 
    markers=True, 
    markersize=8, 
    linewidth=2
)

# Use a log scale for time, as the values vary greatly
g.set_yscale('log')
g.set_title('Execution Time vs. Number of Processes', fontsize=16)
g.set_xlabel('Number of Processes (p)', fontsize=12)
g.set_ylabel('Execution Time (s) [Log Scale]', fontsize=12)
plt.legend(title='N value')
plt.xticks(P_VALUES) # Ensure all p values are shown as ticks
plt.grid(True, which="both", ls="--")

# Save the figure
# plt.savefig('figures/time_comparison.png')
plt.show()

### Figure 2: Speedup Analysis

*(Corresponds to your `fig:speedup`)*

In [ ]:
plt.figure(figsize=(10, 6))
g = sns.lineplot(
    data=df, 
    x='Processes (p)', 
    y='Speedup (S)', 
    hue='N', 
    style='N', 
    markers=True, 
    markersize=8, 
    linewidth=2
)

# Plot the ideal speedup line (y=x)
ideal_x = [min(P_VALUES), max(P_VALUES)]
ideal_y = ideal_x
plt.plot(ideal_x, ideal_y, 'k--', linewidth=2, label='Ideal Speedup')

g.set_title('Speedup vs. Number of Processes', fontsize=16)
g.set_xlabel('Number of Processes (p)', fontsize=12)
g.set_ylabel('Speedup (S)', fontsize=12)
plt.legend(title='N value')
plt.xticks(P_VALUES)
plt.grid(True, which="both", ls="--")

# Save the figure
# plt.savefig('figures/speedup.png')
plt.show()

### Figure 3: Efficiency Analysis

*(Corresponds to your `fig:efficiency`)*

In [ ]:
plt.figure(figsize=(10, 6))
g = sns.lineplot(
    data=df, 
    x='Processes (p)', 
    y='Efficiency (E)', 
    hue='N', 
    style='N', 
    markers=True, 
    markersize=8, 
    linewidth=2
)

# Plot the ideal efficiency line (y=1)
plt.axhline(1.0, color='k', linestyle='--', linewidth=2, label='Ideal Efficiency')

g.set_title('Efficiency vs. Number of Processes', fontsize=16)
g.set_xlabel('Number of Processes (p)', fontsize=12)
g.set_ylabel('Efficiency (E)', fontsize=12)
g.set_ylim(0, 1.1) # Efficiency is between 0 and 1
plt.legend(title='N value', loc='lower left')
plt.xticks(P_VALUES)
plt.grid(True, which="both", ls="--")

# Save the figure
# plt.savefig('figures/efficiency.png')
plt.show()